In [1]:
#functions
import hashlib
import itertools

path = "C:\\Users\\peorr\\OneDrive\\Documentos\\GitHub\\ComputerSecurity\\decrypt"
########### GENERATE KEYS ###############################
def generate_key(k0):
    # Cálculo de k1 como un desplazamiento cíclico de 1 bit a la derecha de k0
    k1 = bytearray([(k0[0] >> 1) | ((k0[1] & 0x01) << 7), (k0[1] >> 1)])

    # Cálculo de los hashes SHA-512 de k0 y k1
    h0 = hashlib.sha512(k0).digest()
    h1 = hashlib.sha512(k1).digest()

    # Concatenar h0 y h1 para formar la clave de 1024 bits (128 bytes)
    key = h0 + h1
    return key.hex()

### One time pad ENCRYPTION 

def decrypt(ciphertext, key):
    # Convert ciphertext and key to binary using hex decoding
    ciphertext_bin = bytes.fromhex(ciphertext)
    key_bin = key.encode('ascii')

    # Compute the plaintext by XORing the ciphertext and key
    plaintext = ''.join(chr(ct_byte ^ key_byte)
                        for ct_byte, key_byte in zip(ciphertext_bin, key_bin))

    return plaintext



def convert_int_tuple_to_string(int_tuple):
    str_list = [str(num) for num in int_tuple]
    string = ''.join(str_list)
    return string

#convert to bytes
def convert_binary_string_to_bytes_and_bytearray(binary_string):
    # Convert binary string to bytes
    byte_data = bytes(int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8))
    
    # Convert bytes to bytearray
    byte_data = bytearray(byte_data)
    
    return byte_data


#########################LANGUAGE PROBABILITIES#########################
#function that only returns the sentences with prob >=0.5 to be english language
from langdetect import detect_langs

def is_english(sentence, threshold=0.5):
    try:
        languages = detect_langs(sentence)
        for lang in languages:
            if lang.lang == 'en' and lang.prob >= threshold:
                return sentence
        return ""
    except:
        return ""
    

In [2]:
#generate all possible 16 bit
combinations = list(map(convert_int_tuple_to_string, itertools.product([0, 1], repeat=16)))
combinations = list(map(convert_binary_string_to_bytes_and_bytearray,combinations))


In [3]:
#generate keys from combinations
keys = list(map(generate_key,combinations))

In [4]:
#open message
with open(path+'/Ciphertext-1.txt', 'r') as f:
    secret= f.readlines()
    f.close()
    #save it

print(secret)


['695c475f545a4419416404551048571011170d580f1414792f1d167100120645075d125854574d0a0c5e18505949154527075717411941020a4d445e5e4a1357444419465b050c51064c4271151643455e5c0b575619405c12035644150e5707531407445f4c12615d04424a094212460a5a5f40']


In [5]:
#generate decryption of message

dec = list(map(lambda x: decrypt(secret[0],x),keys))
# check probbaility of language english

check =list(map(is_english,dec))
#remove non probable
check[:]= (value for value in check if value != "")
#double language check

check2 =list(map(lambda x: is_english(sentence = x,threshold=0.9999),check))
check2[:]= (value for value in check2 if value != "")
# Imprimir el texto descifrado
if len(check2) > 0:
    print("Texto descifrado:")
    for texto_descifrado in check2:
        print(texto_descifrado)

['X8\x7fkgcs(vU=dt|`q  o>6u M\x19~/@3$5re>*`0a.88ozbl("#\x1135.wz%fhz}hir+2&t.pl5:36| D%%{\'l>?ed #kpbgwwke>k&4si}pV<7{smvtto8>w', '\\lqfdct-"\x0254v)bssq?;8",\x1f\x1a~sGeqe|ddsn`6~o48+b;x  B0cuq,%foxskk| `rv-w9co03/!\x13!t{p:?n1b!$>!17",k`d2%0rjt$W;6s/0$*\x7fkmiv', 'Yov>6c"\x7ft]2gupf!".ln6$$K\x1e+/E5tep?is:eo{ki<)2?z!|\x1fda!p{x5<,smgyr1}p{p9foh1tv\x17"s{&<k>5f.q8w5f",h3fc"cpk(v\x05h2s/<qwu9nlx', '^hs>bl!{x\x00<mu\x7f1(#s<:7!$\x1c\x1a* H7vcs6m%n`a+hnh-1hzt}\x1542\'w\x7frck\x7f&jj\x7f"`pt|~>253g({\x17\' %$?hofb\x7f#9#a4% <g0f\'1wf(\'Yo4{(>t+":?kv', '\x08l$n0j\' y\\0er-n"#$ij>pqJ\x17.pB9pd$3ms`a4{34i{5:xw#\x1775 w(tfi~tmg\x7f%4pr+s86j42- B#\'%!:9ne2!x?s0cvt6`>f$arit$Rng!)h w ;;f&', ']jrf18&zuU00"\x7f`&%.:kmwr\x1cN{$\x146pbufd%9c`t?9f!h<x$&\x147cs$xq4:ts;?{+f&p}podoh3{qD p\'&gj3bo}&d#aovp=ab0%c\'m.%Pmg +9#wp;9iu', "\x08lw=f8v*%\x011l(\x7fo)p n=9rpO\x1fx B6t1sek$k7b\x7f>8h*`8(t}A>nv$/%3iytkf/rewt-qm6=f>tv\x17wws|mh8ea)p='5f&'=330qbqk{&Xj7v\x7f8pqvkmmv", '^:$=6br*w\x00aa).5!\'$;`;v%J\x1e+

In [6]:
# get key from encryption
keys[560]

'0b0e93167a2b03369cdb7b1bf5f46758929afcda5bbb63ca4897028650e024440f703418e63d6a59dcc7f57538df08db1aa484619f91c1e7be4784efbd907b158993dd23a402cd3cf6ed8733860d58fbb3ed12a56bcde85c124b629d9d33029881f9e6ea2226d0a374d8d0c0a2f4bcf003ada3662ff25dca81f72b1884e3893f'